In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import collections
from math import sin, cos, sqrt, atan2, radians, asin
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql.functions import col, radians, asin, sin, sqrt, cos
from haversine import haversine
import time

# Create a SparkSession (Note, the config section is only for Windows!)
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "data").appName("SparkSQL").getOrCreate()


rdd = sc.textFile("hotels-exmpl.txt")
rdd1 = rdd.map(lambda x : (x.split("|")[0], x.split("|")[1], x.split("|")[4], x.split("|")[5]))
df1 = rdd1.toDF()
df2 = df1.selectExpr("_1 as _id", "_2 as name","_3 as lat", "_4 as lon")
df3 = df2.withColumn("dataset_id", lit("B"))

rdd2 = sc.textFile("restaurants-exmpl.txt")
rdd3 = rdd2.map(lambda x : (x.split("|")[0], x.split("|")[1], x.split("|")[3], x.split("|")[4]))
df4 = rdd3.toDF()
df5 = df4.selectExpr("_1 as _id", "_2 as name","_3 as lat", "_4 as lon")
df6 = df5.withColumn("dataset_id", lit("A"))

df_concat = df3.union(df6)
df_concat1 = df_concat.orderBy(df_concat["lat"].asc())
dataset = df_concat1.withColumn("increas_id",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)
dataset1 = dataset.withColumn("partition",(col("increas_id")/260).cast("int"))
dataset2 = dataset1.withColumn("lat", dataset["lat"].cast("float"))
dataset3 = dataset2.withColumn("lon", dataset["lon"].cast("float"))
dataset1_partitioned = dataset3.repartitionByRange(4, col("partition"))
#dataset1_partitioned.write.mode("overwrite").csv("apotelesmata/results.txt")

start_time = time.time() 
dist = 2
final_table = []
for x in dataset1_partitioned.collect():
    for y in dataset1_partitioned.collect():
        if (x["dataset_id"] == "A") and (y["dataset_id"] == "B"):
            restaurant_id = x[0]
            restaurant_name = x[1]
            restaurant_lat = x[2]
            restaurant_lon = x[3]
  
            max_distance = 0.5  #10
            R = 6373.0
            hotel_id = y[0]
            hotel_name = y[1]
            hotel_lat = y[2]
            hotel_lon = y[3]
            
            distance = haversine((restaurant_lat, restaurant_lon),(hotel_lat, hotel_lon))

            
            if distance <= max_distance:
                row = []
                row.append(restaurant_id)
                row.append(restaurant_name)
                row.append(hotel_id)
                row.append(hotel_name)
                row.append(np.around(distance,2))
                final_table.append(row)

print("--- %s seconds ---" % (time.time() - start_time)) 

results_DF = pd.DataFrame(final_table,columns=['Restaurant Id', 'Restaurant Name', 'Hotel Id', 'Hotel Name','Distance'])
results_DF.count()
print(results_DF)

--- 87.0589747428894 seconds ---
   Restaurant Id        Restaurant Name Hotel Id  \
0            365          Coco's Bakery      446   
1            365          Coco's Bakery       73   
2            524        Jack in the Box      309   
3            524        Jack in the Box       48   
4            420         Royal Hawaiian      220   
..           ...                    ...      ...   
87           474        Jack in the Box      132   
88           104  The Buckeye Roadhouse       70   
89           122            Dairy Queen      495   
90           506        Jack in the Box      337   
91           201              Carl's Jr      451   

                       Hotel Name  Distance  
0      Best Western Americana Inn      0.02  
1        America's Best Value Inn      0.31  
2                    Best Western      0.36  
3        America's Best Value Inn      0.27  
4                 Arabella Laguna      0.49  
..                            ...       ...  
87       America's B